# Quick Start: Create an Agent (API)

Build your first RAG agent programmatically using the Contextual AI Python SDK.

## Prerequisites
- Python 3.8+
- A Contextual AI account with API key

## Step 1: Install the SDK

In [ ]:
!pip install contextual-client

## Step 2: Set Up Your Environment

Replace `YOUR_API_KEY` with your actual API key from [app.contextual.ai](https://app.contextual.ai) → API Keys

In [ ]:
import os
from contextual import ContextualAI

# Option 1: Set your API key as an environment variable (recommended)
# os.environ["CONTEXTUAL_API_KEY"] = "your-api-key"

# Option 2: Pass it directly
client = ContextualAI(api_key="key-_aI34gjLaudoBmgnX8DRq9ODoL-wRMXpSgaaeU2WhENO7mH7E")

## Step 3: Set Up a Datastore

### Option A: Use a pre-processed sample datastore (fastest)

In [ ]:
# List available datastores
datastores = client.datastores.list()
for ds in datastores.datastores:
    print(f"- {ds.name}: {ds.id}")

In [ ]:
# Pick a datastore ID from above and set it here
datastore_id = "1464f60d-2f66-42c8-aa0c-a7fdfe4b78a6"  # Replace with an actual ID from the list above

### Option B: Upload your own documents

Skip this section if you're using Option A.

In [ ]:
# Create a datastore
datastore = client.datastores.create(name="Test Datastore")
print(f"Datastore created: {datastore.id}")
datastore_id = datastore.id

In [ ]:
# Upload a document
with open("Cookbook_Catalog_example_pages.pdf", "rb") as f:
    result = client.datastores.documents.ingest(
        datastore_id=datastore_id,
        file=f
    )
    print(f"Document uploaded: {result.id}")

In [ ]:
# Check document status (wait for ingestion to complete)
import time

print("Waiting for document to process...")
while True:
    metadata = client.datastores.documents.metadata(
        datastore_id=datastore_id,
        document_id=result.id
    )
    print(f"Status: {metadata.status}")
    if metadata.status == "completed":
        print("Document ready!")
        break
    time.sleep(5)

## Step 4: Create an Agent

In [ ]:
# Create your agent
agent = client.agents.create(
    name="My Elastic Search Agent",
    description="A RAG agent for the Elastic Hack Night",
    datastore_ids=[datastore_id]
)

print(f"Agent created! ID: {agent.id}")

## Step 5: Query Your Agent

In [ ]:
# Query the agent
response = client.agents.query.create(
    agent_id=agent.id,
    messages=[
        {
            "role": "user",
            "content": "Type your query specific to your data source here."
        }
    ]
)

# Print the response
print("Agent Response:")
print(response.message.content)

# Print the sources (attributions)
if response.retrieval_contents:
    print("\nSources:")
    for source in response.retrieval_contents[:3]:  # Show first 3 sources
        content_preview = source.content_text[:100] if source.content_text else "(no content preview)"
        print(f"- {source.doc_name} (page {source.page}, score: {source.score:.2f}): {content_preview}...")

## Step 6: Customize Your Agent's Prompts (Optional)

In [ ]:
# Update agent with custom prompts
agent_id = agent.id  # Save the agent ID before update

client.agents.update(
    agent_id=agent_id,
    system_prompt="""You are a helpful search assistant powered by Contextual AI and Elasticsearch.

Your personality:
- Friendly and professional
- Concise but thorough
- Always cite your sources
- Begin each response with "Ahoy!"

When answering questions:
1. Search the datastore for relevant information
2. Provide a clear, direct answer
3. Reference the source documents
"""
)

print("Agent prompts updated!")

## Step 7: Test with Different Queries

In [ ]:
queries = [
    "Summarize the main topics covered in these documents",
    "What are the key findings?",
    "Are there any technical specifications mentioned?"
]

for query in queries:
    print(f"\n{'='*50}")
    print(f"Query: {query}")
    print('='*50)
    
    response = client.agents.query.create(
        agent_id=agent_id,
        messages=[{"role": "user", "content": query}]
    )
    
    print(response.message.content[:500])  # Print first 500 chars

## Bonus: Multi-Turn Conversations

In [ ]:
# First message
response1 = client.agents.query.create(
    agent_id=agent_id,
    messages=[{"role": "user", "content": "What topics are covered?"}]
)
conversation_id = response1.conversation_id
print("First response:")
print(response1.message.content)

# Follow-up message (agent remembers context)
response2 = client.agents.query.create(
    agent_id=agent_id,
    conversation_id=conversation_id,
    messages=[{"role": "user", "content": "Tell me more about the first one"}]
)
print("\nFollow-up response:")
print(response2.message.content)

## Step 8: Submit Your Completion

You did it! Now submit your work:

1. Take a screenshot of any query response from this notebook showing a successful query
2. Fill out the [submission form](https://docs.google.com/forms/d/e/1FAIpQLSdb941rSlQ6s4EcYMu-OwiLkC_jeI-8fD181pYnsbddQnNfTA/viewform) with:
   - Your email address
   - Your screenshot
   - (Optional) Did you use GUI or API?